In [1]:
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import ast 
import nltk
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import pandas as pd
import torch
from transformers import BertTokenizer, BertForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, ClassLabel, Sequence
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
!pip install "transformers[torch]" -U
!pip install accelerate -U

### Reading in Data ###

In [3]:
df = pd.read_csv('final_df.csv')

df['preprocessed_receipt'] = df['preprocessed_receipt'].apply(ast.literal_eval)
df['bio_tags'] = df['bio_tags'].apply(ast.literal_eval)
df['tokenized_receipt'] = df['tokenized_receipt'].apply(ast.literal_eval)

df.head()

15662


,venue_id,receipt_key,receipt_file,tokenized_receipt,preprocessed_receipt,parsed_receipt,bio_tags
0,5531,5531/01/0000000036812256/202003181643-u1584546...,*_*_*_*_*_*_*_*_*_*_*_*_*_*\nThis receipt is p...,"[[*_*_*_*_*_*_*_*_*_*_*_*_*_*], [This, receipt...",[[*_*_*_*_*_*_*_*_*_*_*_*_*_*]],NaN,[[O]]
1,5531,5531/01/0000000036812256/202005150714-u1589519...,*_*_*_*_*_*_*_*_*_*_*_*_*_*\nThis receipt is p...,"[[*_*_*_*_*_*_*_*_*_*_*_*_*_*], [This, receipt...",[[*_*_*_*_*_*_*_*_*_*_*_*_*_*]],NaN,[[O]]
2,5531,5531/01/0000000036812256/202005151410-u1589544...,*_*_*_*_*_*_*_*_*_*_*_*_*_*\nThis receipt is p...,"[[*_*_*_*_*_*_*_*_*_*_*_*_*_*], [This, receipt...",[[*_*_*_*_*_*_*_*_*_*_*_*_*_*]],NaN,[[O]]
3,5531,5531/01/0000000036812256/202005152249-u1589575...,*_*_*_*_*_*_*_*_*_*_*_*_*_*\nThis receipt is p...,"[[*_*_*_*_*_*_*_*_*_*_*_*_*_*], [This, receipt...",[[*_*_*_*_*_*_*_*_*_*_*_*_*_*]],NaN,[[O]]
4,5531,5531/01/0000000036812256/202005180927-u1589786...,*_*_*_*_*_*_*_*_*_*_*_*_*_*\nThis receipt is p...,"[[*_*_*_*_*_*_*_*_*_*_*_*_*_*], [This, receipt...",[[*_*_*_*_*_*_*_*_*_*_*_*_*_*]],NaN,[[O]]


In [4]:
df = df.dropna()
data = df
data.reset_index(drop=True, inplace=True)
df.head()

,venue_id,receipt_key,receipt_file,tokenized_receipt,preprocessed_receipt,parsed_receipt,bio_tags
0,5531,5531/01/0000000036812256/202007161954-u1594922...,**start** \n--...,"[[**start**], [--paid--], [Table:, 2], [------...","[[1, x, Thee, =>, 2,20], [3, x, Jus, d'Orange,...",Thee x 1 order 220 Notes: _Jus d'Orange x 3 or...,"[[B-QTY, O, B-UNIT, O, B-PRICE], [B-QTY, O, B-..."
1,5531,5531/01/0000000036812256/202007162054-u1594925...,**start** \n--...,"[[**start**], [--paid--], [Table:, 7], [------...","[[2, x, Gregorius, =>, 15,90], [2, x, Grimberg...",Gregorius x 2 order 1590 Notes: _Grimbergen Bl...,"[[B-QTY, O, B-UNIT, O, B-PRICE], [B-QTY, O, B-..."
2,5531,5531/01/0000000036812256/202007162154-u1594929...,**start** \n--...,"[[**start**], [--paid--], [Table:, 2], [------...","[[1, x, Lipton, Ice-Tea, =>, 2,70], [2, x, Sou...",Lipton Ice-Tea x 1 order 270 Notes: _Sourcy Ro...,"[[B-QTY, O, B-UNIT, I-UNIT, O, B-PRICE], [B-QT..."
3,5531,5531/01/0000000036812256/202007162254-u1594932...,**start** \n--...,"[[**start**], [--paid--], [Table:, 7], [------...","[[6, x, Pico, de, Aneto, =>, 25,50], [1, x, Ru...",Pico de Aneto x 6 order 2550 Notes: _Rundvlees...,"[[B-QTY, O, B-UNIT, I-UNIT, I-UNIT, O, B-PRICE..."
4,5531,5531/01/0000000036812256/202007162254-u1594932...,**start** \n--...,"[[**start**], [--paid--], [Table:, 2], [------...","[[1, x, Pinot, Grigio, Torre, Dei, V, =>, 4,50...",Pinot Grigio Torre Dei V x 1 order 450 Notes: ...,"[[B-QTY, O, B-UNIT, I-UNIT, I-UNIT, I-UNIT, I-..."


In [5]:
def get_new_bio_tags(tokenized_receipt, preprocessed_receipt, bio_tags):
    new_tags = []
    preprocessed_receipt_tuples = [tuple(line) for line in preprocessed_receipt]
    
    tag_dict = dict(zip(preprocessed_receipt_tuples, bio_tags))
    for line in tokenized_receipt:
        line_tuple = tuple(line)
        if line not in preprocessed_receipt:
            line_length = len(line)
            o_line = ['O'] * line_length
            new_tags.append(o_line)
        else:
            new_tags.append(tag_dict[line_tuple])
            
    return new_tags


[['O'], ['O'], ['O', 'O'], ['O'], ['B-QTY', 'O', 'B-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'O', 'B-PRICE'], ['O'], ['O'], ['O'], ['O', 'O', 'O'], ['O'], ['O', 'O', 'O'], ['O', 'O', 'O'], ['O'], ['O', 'O'], ['O'], ['O']]
[['**start**'], ['--paid--'], ['Table:', '2'], ['------------------------------------------'], ['1', 'x', 'Thee', '=>', '2,20'], ['3', 'x', 'Jus', "d'Orange",

In [6]:
df['bio_tags'] = df.apply(lambda row: get_new_bio_tags(row['tokenized_receipt'], row['preprocessed_receipt'], row['bio_tags']), axis=1)
df.loc[1234,'tokenized_receipt']

[['**start**'],
 ['--paid--'],
 ['Table:', '1'],
 ['------------------------------------------'],
 ['2', 'x', 'Karmeliet', '=>', '11,00'],
 ['------------------------------------------'],
 ['11,00'],
 ['------------------------------------------'],
 ['Contant', '=>', '11,00'],
 ['------------------------------------------'],
 ['21%', '=>', '1,91'],
 ['------------------------------------------'],
 ['01.08.2020', '16:27'],
 ['**end**'],
 ['V\x01']]

In [8]:
lines = [line for receipt in df['tokenized_receipt'] for line in receipt]
tags = [tag for receipt in df['bio_tags'] for tag in receipt]
print(tags[243])

['B-QTY', 'O', 'B-UNIT', 'I-UNIT', 'I-UNIT', 'O', 'B-PRICE']


### Data Preprocessing ###

In [ ]:
flattened_tokens = [token for receipt in df['tokenized_receipt'] for token in receipt]
flattened_tags = [tag for tags in df['bio_tags'] for tag in tags]

data = {"tokens": flattened_tokens, "tags": flattened_tags}
dataset = Dataset.from_dict(data)

tag_names = ['B-QTY', 'B-NOTE', 'I-UNIT', 'O', 'B-PRICE', 'B-UNIT', 'I-NOTE']
tag2id = {tag: id for id, tag in enumerate(tag_names)}
id2tag = {id: tag for tag, id in tag2id.items()}

def encode_tags(tags):
    return [tag2id[tag] for tag in tags]

dataset = dataset.map(lambda x: {"tags": encode_tags(x["tags"])})

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]



In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label[word_idx] != -100 else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

train_dataset = train_dataset.remove_columns(["tokens", "tags"])
test_dataset = test_dataset.remove_columns(["tokens", "tags"])



### Model Training ###

In [ ]:
from transformers import BertForTokenClassification, TrainingArguments, Trainer

model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(tag_names))

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

trainer.train()
